In [108]:
# these alignments aren't great. going to try this again when i have ibm model

In [1]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

In [2]:
def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

In [3]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

In [46]:
f = open('../../Latin/Horace/opensource/hor.ap_lat.xml')
ltext = f.read()
f.close()
f = open('../../Latin/Horace/opensource/hor.ap_eng.xml')
etext = f.read()
f.close()

lat_sections = re.split(r'<milestone ed="p" n="\d\d?\d?" unit="card"/>', ltext)

eng_sections = re.split(r'<milestone ed="p" n="\d\d?\d?" unit="card"/>', etext)

In [47]:
len(lat_sections)

15

In [48]:
len(eng_sections)

14

In [49]:
parts = eng_sections[5].split('Do you attend to what I')

In [50]:
part_1 = parts[0]

In [51]:
part_2 = 'Do you attend to what I'+parts[1]

In [52]:
eng_sections = eng_sections[:5] + [part_1] + [part_2] + eng_sections[6:]

In [53]:
len(lat_sections)

15

In [54]:
len(eng_sections)

15

In [66]:
lat_text[0]

"Humano capiti cervicem pictor equinam \niungere si velit, et varias inducere plumas, \nundique conlatis membris, ut turpiter atrum \ndesinat in piscem mulier formosa superne, \nspectatum admissi risum teneatis, amici? \ncredite, Pisones, isti tabulae fore librum \npersimilem, cuius, velut aegri somnia, vanae \nfingentur species, ut nec pes nec caput uni \nreddatur formae. 'pictoribus atque poetis \nquidlibet audendi semper fuit aequa potestas.' \nscimus, et hanc veniam petimusque damusque vicissim; \nsed non ut placidis coeant inmitia, non ut  \nserpentes avibus geminentur, tigribus agni. \ninceptis gravibus plerumque et magna professis \npurpureus, late qui splendeat, unus et alter \nadsuitur pannus, cum lucus et ara Dianae \net properantis aquae per amoenos ambitus agros \naut flumen Rhenum aut pluvius describitur arcus; \nsed nunc non erat his locus. et fortasse cupressum \nscis simulare: quid hoc, si fractis enatat exspes \nnavibus, aere dato qui pingitur? amphora coepit \ninstitu

In [103]:

eng_text = []

for s in eng_sections:
    if re.match(r'</.>',s):
        s = re.split(r'</.>',s,maxsplit=1)[1]
    soup = BeautifulSoup(s,'lxml')
    if soup.find('note'):
        s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
        soup = BeautifulSoup(s,'lxml')
        eng_text.append(soup.text)
    else:
        eng_text.append(soup.text)

lat_text = []

for s in lat_sections:
    if re.match(r'</.>',s):
        s = re.split(r'</.>',s,maxsplit=1)[1]
    soup = BeautifulSoup(s,'lxml')
    if soup.find('note'):
        s = re.sub(r'<note>.*</note>', " ", s,flags=re.DOTALL)
        soup = BeautifulSoup(s,'lxml')
        lat_text.append(soup.text)
    else:
        lat_text.append(soup.text)

eng_text = eng_text[1:]
lat_text = lat_text[1:]

alignments = []

for c in range(len(eng_text)):
    es = ' '.join(eng_text[c].replace('\n',' ').split())
    ls = lat_text[c].replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub('[:;,]','',ls)
    es = re.sub('[:;,]','',es)

    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

In [104]:
len(alignments)

14

In [107]:
alignments[5]

[("tu quid ego et populus mecum desideret audi  si plausoris eges aulaea manentis et usque  sessuri donec cantor 'vos plaudite' dicat.",
  ['Do you attend to what I and the public in my opinion expect from you [as a dramatic writer].']),
 ('aetatis cuiusque notandi sunt tibi mores  mobilibusque decor naturis dandus et annis.',
  ['If you are desirous of an applauding spectator who will wait for [the falling of] the curtain and till the chorus calls out "your plaudits" the manners of every age must be marked by you and a proper decorum assigned to men\'s varying dispositions and years.']),
 ('reddere qui voces iam scit puer et pede certo  signat humum gestit paribus conludere et iram  colligit ac ponit temere et mutatur in horas.',
  ['The boy who is just able to pronounce his words and prints the ground with a firm tread delights to play with his fellows and contracts and lays aside anger without reason and is subject to change every hour.']),
 ('inberbis iuvenis tandem custode remoto 

In [83]:
soup = BeautifulSoup(e,'lxml')

In [82]:
e = eng_sections[-1].replace('</p>\n', '')

In [102]:
re.split(r'</.>',eng_sections[2],maxsplit=1)

['',
 '\n<p>In the choice of his words, too, the author of the projected poem must be delicate and cautious, he must embrace one and reject another: you will express yourself eminently well, if a dexterous combination should give an air of novelty to a wellknown word. If it happen to be necessary to explain some abstruse subjects by new invented terms; it will follow that you must frame words never heard of by the old-fashioned<note anchored="yes" n="8" resp="McCaul"><p><cit><quote lang="la">Cinctutis.</quote><bibl n="Hor. Ars 50"></bibl></cit> Having the tunic tightened by the cinctus, or wearing the cinctus instead of the tunic, as appears to have been the custom of the ancient Romans. This was a vest which passed round the waist, and extended down to the feet. That it was an ancient vesture may appear from its being used by the Luperci. Comp. <bibl n="Ov. Fast. 5.101">Ovid. Fast. v. 101</bibl>. As it did not embarrass the motion of the arms, even after the tunic became part of the d

In [98]:
eng_text[0]

'If a painterAll that our poet says here may be referred, in general, to three heads, the fable, the manners, and the diction. We should take notice that this piece particularly regards epic and dramatic poetry, and that our author only occasionally mentions any other kind. The most important precept for the composition of a poem is unity and simplicity of design. There should be only one action, to which all the incidents ought to refer; and this point of perfection, every regular work requires. To show the necessity of this rule, Horace compares an irregular poem to pictures formed by a wild assortment of many parts entirely unlike each other. Every part, considered in itself, may have its proper, natural perfection, while their union produces nothing but what is monstrous and ridiculous. FRAN. The critic\'s rules must be taken either, 1. from the general standing laws of composition; or, 2. from the peculiar ones, appropriated to the kind. Now the direction to be fetched from the fo